In [ ]:
!pip install transformers datasets accelerate
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
with open('10.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()

In [ ]:
def split_text_into_chunks(text, max_chunk_chars):
    import textwrap
    chunks = textwrap.wrap(text, max_chunk_chars, break_long_words=False, break_on_hyphens=False)
    print(f"Total chunks created: {len(chunks)}\n")
    print("Sample chunk: ")
    print(chunks[8])
    return chunks

chunks = split_text_into_chunks(full_text, max_chunk_chars=2000)

Total chunks created: 41

Sample chunk: 
shall be levied and collected in  accordance with the provisions of section 3 of the Customs Tariff Act, 1975, at the point when  duties of customs are levied on the said goods under section 12 of the Customs Act,1962, on a  value determined under the Customs Tariff Act, 1975.  Section 9. Returns, payments and refunds.    Every taxable person, making a taxable supply of goods or services or both, shall-    pay the amount of cess as payable under this Act in such manner;    furnish such returns in such forms, along with the returns to be filed under the Central  Goods and Services Tax Act; and    apply for refunds of such cess paid in such form, as may be prescribed.    The Goods and Services Tax  Act, 2017  Sec 1-14    380  GST & Indirect Taxes Committee     For all purposes of furnishing of returns and claiming refunds, except for the form to be filed,  the provisions of the Central Goods and Services Tax Act and the rules made there under, sha

In [ ]:
from groq import Groq
import time

client = Groq(api_key= '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')  # Replace with your key

qa_pairs = []

for idx, chunk in enumerate(chunks):  # Adjust chunk limit as needed
    print(f"Processing chunk {idx + 1} of {len(chunks)}")
    # Step 4.1: Generate 5 questions from this chunk
    try:
        messages = [
            {"role": "system", "content": "You are being used to generate a dataset for finrtuning a llm a Indian GST law chatbot."},
            {"role": "user", "content": f"Generate 5 unique and insightful questions based on the following legal text that a user may ask about Indian GST law:\n\n{chunk}\n The question should look like a query posed by a user to the GST chatbot. Do not  use meta-explanations (e.g., This question requires the chatbot to...) as the question."}
        ]
        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=messages,
            temperature=0.7,
            max_tokens=400
        )
        questions_text = response.choices[0].message.content.strip()
        questions = [q.strip("-•1234567890. ").strip() for q in questions_text.split("\n") if q.strip()]
        questions = questions[1:]  # Keep only the first 5 questions
    except Exception as e:
        print(f"[QGen Error] at chunk {idx}: {e}")
        continue

    # Step 4.2: Get answers for each question
    for question in questions:
        try:
            messages = [
                {"role": "system", "content": "I want you to answer the following question on Indian GST law to create answers for a dataset for finetuning an llm. just provide th answer, do not include any comments like 'based on the context provided', just give the outright answer. Do not give answers that are too short or vague. Avoid inconsistent use of \n for bullet points unless you're using proper markdown formatting throughout."},
                {"role": "user", "content": f"Question: {question}\n\nRefer to this legal content:\n{chunk}"}
            ]
            response = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=messages,
                temperature=0.7,
                max_tokens=300
            )
            answer = response.choices[0].message.content.strip()
            qa_pairs.append({
                "question": question,
                "answer": answer
            })
            time.sleep(0.3)
        except Exception as e:
            print(f"[Answer Error] for question '{question[:30]}': {e}")

Processing chunk 1 of 41
Processing chunk 2 of 41
Processing chunk 3 of 41
Processing chunk 4 of 41
Processing chunk 5 of 41
Processing chunk 6 of 41
Processing chunk 7 of 41
Processing chunk 8 of 41
Processing chunk 9 of 41
Processing chunk 10 of 41
Processing chunk 11 of 41
Processing chunk 12 of 41
Processing chunk 13 of 41
Processing chunk 14 of 41
Processing chunk 15 of 41
Processing chunk 16 of 41
Processing chunk 17 of 41
Processing chunk 18 of 41
Processing chunk 19 of 41
Processing chunk 20 of 41
Processing chunk 21 of 41
Processing chunk 22 of 41
Processing chunk 23 of 41
Processing chunk 24 of 41
Processing chunk 25 of 41
Processing chunk 26 of 41
Processing chunk 27 of 41
Processing chunk 28 of 41
Processing chunk 29 of 41
Processing chunk 30 of 41
Processing chunk 31 of 41


In [ ]:
# STEP 5: Create and Save the Dataset
from datasets import Dataset

dataset = Dataset.from_list(qa_pairs)
dataset.to_json("output10.json")

# STEP 6: Download the file
# files.download("gst_qa_dataset.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

126354

In [ ]:
from google.colab import files

files.download("output10.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>